In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

df = pd.read_csv('2021 END OF YEAR ROUND-UP_ South Florida Hotel Sales - Final.csv')

### Create clean CSV for export for graphics data
Bascially, rename the columns so they aren't fussy.

In [5]:
df2 = pd.read_csv('2021 END OF YEAR ROUND-UP_ South Florida Hotel Sales - Final.csv')
df2.columns = df2.columns.str.replace(' ','')
df2.columns = df2.columns.str.replace('/','_')
df2.drop('#', axis=1, inplace=True)
df2 = df2.rename(columns={'Size(SF)':'Square_Footage'})
df2.to_csv('2021_hotel_sales_for_graphic.csv',index=False)
df2.columns

Index(['PropertyName', 'PropertyAddress', 'City', 'SalePrice',
       'Square_Footage', 'Units', 'Price_SF', 'Price_Unit', 'SaleDate',
       'Buyer', 'Seller', 'Link'],
      dtype='object')

### --- All done. Return to map making ---

In [121]:
df.at[0,'Property Address']=('1111 North Ocean Drive, Hollywood, FL')

In [122]:
df

,#,Property Name,Property Address,City,Sale Price,Size (SF),Units,Price/SF,Price/Unit,Sale Date,Buyer,Seller,Link
0,1,Margaritaville Hollywood Beach Resort,"1111 North Ocean Drive, Hollywood, FL",Hollywood,"$270,000,000","666,300",369,$405.22,"$731,707",Sep-21,Pebblebrook Hotel Trust,KSL Capital Partners,https://www.bizjournals.com/southflorida/news/...
1,2,EAST Miami,788 Brickell Plz,Miami,"$174,000,000","536,070",352,$324.58,"$494,318",Oct-21,Trinity Investments,Swire Properties,https://www.bizjournals.com/southflorida/news/...
2,3,Four Seasons,1441 Brickell Ave,Miami,"$130,000,000","263,867",221,$492.67,"$588,235",Jul-21,Fort Partners,Westbrook Partners,https://therealdeal.com/miami/2021/07/15/fort-...
3,4,B Ocean Resort,1140 Seabreeze Blvd,Fort Lauderdale,"$126,875,000","368,646",481,$344.16,"$263,773",Oct-21,JV: Rockpoint Group & InSite Group,Carlyle Group,https://www.bizjournals.com/southflorida/news/...
4,5,The Ben,251 N Narcissuc Ave,West Palm Beach,"$106,400,000","146,746",208,$725.06,"$511,538",Oct-21,Wheelock Street Capital,Concord Hospitality,https://therealdeal.com/miami/2021/10/08/wheel...
5,6,Celino South Beach,640 Ocean Dr,Miami Beach,"$81,453,800","59,359",132,"$1,372.22","$617,074",Jul-21,CGI Merchant Group,Optimum Asset Management,https://therealdeal.com/miami/2021/08/05/hospi...
6,7,Hotel AKA Brickell,1395 Brickell Ave,Miami,"$75,330,000","212,503",201,$354.49,"$374,776",Oct-21,JV: Electra America & AKA,JV: Mast Capital & Angelo Gordon,https://www.bizjournals.com/southflorida/news/...
7,8,Como Metropolitan Miami Beach,2445 Collins Ave,Miami Beach,"$70,000,000","53,328",74,"$1,312.63","$945,946",Aug-21,JHSF,COMO Hotels and Resorts,https://www.bizjournals.com/southflorida/news/...
8,9,Hampton Inn & Suites and AC Hotel by Marriott ...,3450 & 3400 Biscayne Blvd,Miami,"$61,000,000","280,000",304,$217.86,"$200,658",Jun-21,TPG Real Estate Partners,3H Group Hotels,https://www.bizjournals.com/southflorida/news/...
9,10,Fort Lauderdale Marriott Pompano Beach,1200 N Ocean Blvd,Pompano Beach,"$54,000,000","184,184",219,$293.19,"$246,575",Dec-21,JV: Key International & Wexford Real Estate In...,Pan Am Equities,https://www.bizjournals.com/southflorida/news/...


In [123]:
df['geo_address'] = df['Property Address']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
# df.at[1,'geo_address']='1111 North Ocean Drive, Hollywood, FL'
df = df.replace('\n',' ',regex=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [124]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [125]:
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)

In [126]:
df.head(1)

,#,Property Name,Property Address,City,Sale Price,Size (SF),Units,Price/SF,Price/Unit,Sale Date,Buyer,Seller,Link,geo_address,loc
0,1,Margaritaville Hollywood Beach Resort,"1111 North Ocean Drive, Hollywood, FL",Hollywood,"$270,000,000","666,300",369,$405.22,"$731,707",Sep-21,Pebblebrook Hotel Trust,KSL Capital Partners,https://www.bizjournals.com/southflorida/news/...,"1111 North Ocean Drive, Hollywood, FL","(1111 N Ocean Dr, Hollywood, FL 33019, USA, (2..."


In [127]:
df['loc']

0    (1111 N Ocean Dr, Hollywood, FL 33019, USA, (2...
1    (788 Brickell Plaza, Miami, FL 33131, USA, (25...
2    (1441 Brickell Ave, Miami, FL 33131, USA, (25....
3    (1140 Seabreeze Blvd, Fort Lauderdale, FL 3331...
4    (251 N Narcissus Ave, West Palm Beach, FL 3340...
5    (640 Ocean Dr, Miami Beach, FL 33139, USA, (25...
6    (Espírito Santo Plaza, 1395 Brickell Ave, Miam...
7    (2445 Collins Ave, Miami Beach, FL 33140, USA,...
8    (3400 Biscayne Blvd #3450, Miami, FL 33137, US...
9    (1200 N Ocean Blvd, Pompano Beach, FL 33062, U...
Name: loc, dtype: object

In [128]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [129]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,#,Property Name,Property Address,City,Sale Price,Size (SF),Units,Price/SF,Price/Unit,Sale Date,Buyer,Seller,Link,geo_address,loc,point,lat,lon,altitude
0,1,Margaritaville Hollywood Beach Resort,"1111 North Ocean Drive, Hollywood, FL",Hollywood,"$270,000,000","666,300",369,$405.22,"$731,707",Sep-21,Pebblebrook Hotel Trust,KSL Capital Partners,https://www.bizjournals.com/southflorida/news/...,"1111 North Ocean Drive, Hollywood, FL","(1111 N Ocean Dr, Hollywood, FL 33019, USA, (2...","(26.0193254, -80.1156831, 0.0)",26.019325,-80.115683,0.0
1,2,EAST Miami,788 Brickell Plz,Miami,"$174,000,000","536,070",352,$324.58,"$494,318",Oct-21,Trinity Investments,Swire Properties,https://www.bizjournals.com/southflorida/news/...,788 Brickell Plz,"(788 Brickell Plaza, Miami, FL 33131, USA, (25...","(25.7666736, -80.1927482, 0.0)",25.766674,-80.192748,0.0
2,3,Four Seasons,1441 Brickell Ave,Miami,"$130,000,000","263,867",221,$492.67,"$588,235",Jul-21,Fort Partners,Westbrook Partners,https://therealdeal.com/miami/2021/07/15/fort-...,1441 Brickell Ave,"(1441 Brickell Ave, Miami, FL 33131, USA, (25....","(25.7581672, -80.1925132, 0.0)",25.758167,-80.192513,0.0
3,4,B Ocean Resort,1140 Seabreeze Blvd,Fort Lauderdale,"$126,875,000","368,646",481,$344.16,"$263,773",Oct-21,JV: Rockpoint Group & InSite Group,Carlyle Group,https://www.bizjournals.com/southflorida/news/...,1140 Seabreeze Blvd,"(1140 Seabreeze Blvd, Fort Lauderdale, FL 3331...","(26.1081401, -80.10625920000001, 0.0)",26.108140,-80.106259,0.0
4,5,The Ben,251 N Narcissuc Ave,West Palm Beach,"$106,400,000","146,746",208,$725.06,"$511,538",Oct-21,Wheelock Street Capital,Concord Hospitality,https://therealdeal.com/miami/2021/10/08/wheel...,251 N Narcissuc Ave,"(251 N Narcissus Ave, West Palm Beach, FL 3340...","(26.7148159, -80.0508291, 0.0)",26.714816,-80.050829,0.0
5,6,Celino South Beach,640 Ocean Dr,Miami Beach,"$81,453,800","59,359",132,"$1,372.22","$617,074",Jul-21,CGI Merchant Group,Optimum Asset Management,https://therealdeal.com/miami/2021/08/05/hospi...,640 Ocean Dr,"(640 Ocean Dr, Miami Beach, FL 33139, USA, (25...","(25.7763776, -80.1319108, 0.0)",25.776378,-80.131911,0.0
6,7,Hotel AKA Brickell,1395 Brickell Ave,Miami,"$75,330,000","212,503",201,$354.49,"$374,776",Oct-21,JV: Electra America & AKA,JV: Mast Capital & Angelo Gordon,https://www.bizjournals.com/southflorida/news/...,1395 Brickell Ave,"(Espírito Santo Plaza, 1395 Brickell Ave, Miam...","(25.7604836, -80.1909065, 0.0)",25.760484,-80.190906,0.0
7,8,Como Metropolitan Miami Beach,2445 Collins Ave,Miami Beach,"$70,000,000","53,328",74,"$1,312.63","$945,946",Aug-21,JHSF,COMO Hotels and Resorts,https://www.bizjournals.com/southflorida/news/...,2445 Collins Ave,"(2445 Collins Ave, Miami Beach, FL 33140, USA,...","(25.8004209, -80.126164, 0.0)",25.800421,-80.126164,0.0
8,9,Hampton Inn & Suites and AC Hotel by Marriott ...,3450 & 3400 Biscayne Blvd,Miami,"$61,000,000","280,000",304,$217.86,"$200,658",Jun-21,TPG Real Estate Partners,3H Group Hotels,https://www.bizjournals.com/southflorida/news/...,3450 & 3400 Biscayne Blvd,"(3400 Biscayne Blvd #3450, Miami, FL 33137, US...","(25.808867, -80.1899911, 0.0)",25.808867,-80.189991,0.0
9,10,Fort Lauderdale Marriott Pompano Beach,1200 N Ocean Blvd,Pompano Beach,"$54,000,000","184,184",219,$293.19,"$246,575",Dec-21,JV: Key International & Wexford Real Estate In...,Pan Am Equities,https://www.bizjournals.com/southflorida/news/...,1200 N Ocean Blvd,"(1200 N Ocean Blvd, Pompano Beach, FL 33062, U...","(26.2491187, -80.0860032, 0.0)",26.249119,-80.086003,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [130]:
df = df.rename(columns={'Size (SF)':'Square Footage','Sale Price ':'Sale Price','Price/SF':'Price/SF'})
df.columns

Index(['#', 'Property Name', 'Property Address', 'City', 'Sale Price',
       'Square Footage', 'Units', 'Price/SF', 'Price/Unit', 'Sale Date',
       'Buyer', 'Seller', 'Link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [131]:
df['Price/SF'] = df['Price/SF'].replace(' ','',regex=True)

In [134]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=8)
# folium.TileLayer('stamentoner').add_to(m)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'2021 South Florida Priciest Hotel Sales')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('The most expensive hotel sales in South Florida')


marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Property Name"], r["Property Address"], r["City"], r["Square Footage"], r["Sale Price"], r["Price/SF"], r["Sale Date"], r["Buyer"], r["Seller"], r["Units"], r["Price/Unit"])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Property Name: {bold_start}{info[0]}{bold_end}{new_line}\
    Property Address: {bold_start}{info[1]}{bold_end}{new_line}\
    Buyer: {bold_start}{info[7]}{bold_end}{new_line}\
    Seller: {bold_start}{info[8]}{bold_end}{new_line}\
    Units: {bold_start}{info[9]}{bold_end}{new_line}\
    Price/Unit: {bold_start}{info[10]}{bold_end}{new_line}\
    City: {bold_start}{info[2]}{bold_end}{new_line}\
    Square Footage: {bold_start}{info[3]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[4]}{bold_end}{new_line}\
    Price/SF: {bold_start}{info[5]}{bold_end}{new_line}\
    Sale Date: {bold_start}{info[6]}{bold_end}{new_line}'
    
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
m

In [135]:
m.save('index.html')